In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.log_softmax(x, dim=1)
    
model = Net()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

In [5]:
epochs = 15
for epoch in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print(f"Epoch {epoch+1} - Training loss: {running_loss/len(trainloader)}")

Epoch 1 - Training loss: 0.6199854300188612
Epoch 2 - Training loss: 0.2775618942029504
Epoch 3 - Training loss: 0.2129518067293457
Epoch 4 - Training loss: 0.17068282111700792
Epoch 5 - Training loss: 0.1414774147841309
Epoch 6 - Training loss: 0.12033996109102073
Epoch 7 - Training loss: 0.10677135018492813
Epoch 8 - Training loss: 0.09453347056055628
Epoch 9 - Training loss: 0.08465910082889487
Epoch 10 - Training loss: 0.07773018521028383
Epoch 11 - Training loss: 0.0706453015695193
Epoch 12 - Training loss: 0.06438403222799254
Epoch 13 - Training loss: 0.05935326086253976
Epoch 14 - Training loss: 0.05364021582917165
Epoch 15 - Training loss: 0.04980829295574197


In [6]:
correct_count, all_count = 0, 0
for images,labels in testloader:
  for i in range(len(labels)):
    with torch.no_grad():
        logps = model(images[i].view(1, 28*28))

    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9753
